#Carregar a base de dados de treino
Você pode usar a biblioteca tensorflow ou keras.preprocessing.image para carregar e pré-processar as imagens, garantindo que elas tenham o formato correto (250x250) antes de realizar a separação dos pacientes.

In [2]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
# Caminho para as imagens de treino
train_dir = 'path/to/train/directory'

# Função para carregar as imagens e seus rótulos
def load_images(train_dir):
    images = []
    labels = []
    for label in os.listdir(train_dir):
        label_dir = os.path.join(train_dir, label)
        for img_file in os.listdir(label_dir):
            img = load_img(os.path.join(label_dir, img_file), target_size=(250, 250))
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(int(label))
    return np.array(images), np.array(labels)


# 2. Separar por paciente (80% treino e 20% validação)
Você pode utilizar a nomenclatura das imagens (XX) para garantir que as partições sejam feitas corretamente por pacientes e evitar o viés.

In [ ]:
# Separação por pacientes (XX)
def split_by_patient(images, labels):
    # Assumindo que XX é a parte inicial do nome da imagem
    patients = [img_file.split('_')[0] for img_file in images]
    unique_patients = np.unique(patients)
    train_patients, val_patients = train_test_split(unique_patients, test_size=0.2, random_state=42)

    train_idx = [i for i, patient in enumerate(patients) if patient in train_patients]
    val_idx = [i for i, patient in enumerate(patients) if patient in val_patients]

    return images[train_idx], labels[train_idx], images[val_idx], labels[val_idx]

# 3. Extração de características usando LBP e CNN VGG16
LBP (Local Binary Patterns) pode ser feito com scikit-image.

VGG16: Utilize a VGG16 pré-treinada, remova a última camada e capture os valores da penúltima camada.

In [ ]:
from skimage.feature import local_binary_pattern

# Parâmetros do LBP
radius = 1
n_points = 8 * radius

# Extração de características com LBP
def extract_lbp(images):
    lbp_features = []
    for img in images:
        gray_img = rgb2gray(img)  # Converter para escala de cinza
        lbp = local_binary_pattern(gray_img, n_points, radius, method='uniform')
        lbp_features.append(lbp.flatten())  # Transformar em vetor
    return np.array(lbp_features)

VGG16:

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

# Carregar o modelo VGG16 pré-treinado e remover a última camada
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)

# Redimensionar as imagens e extrair características
def extract_vgg16(images):
    resized_images = [resize(img, (224, 224)) for img in images]
    features = model.predict(np.array(resized_images))
    return features


# 4. Treinar Random Forest, SVM e RNA
Após a extração de características, você pode treinar modelos de aprendizado de máquina, como Random Forest, SVM e RNA.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# Treinamento dos modelos
def train_models(train_features, train_labels):
    rf = RandomForestClassifier().fit(train_features, train_labels)
    svm = SVC().fit(train_features, train_labels)
    mlp = MLPClassifier().fit(train_features, train_labels)
    return rf, svm, mlp


# 5. Carregar a base de teste e repetir a extração de características
Utilize a mesma abordagem usada no conjunto de treino para extrair as características do conjunto de teste.
# 6. Aplicar os modelos treinados nos dados de teste
Faça as predições usando os modelos treinados nos dados de teste.



In [ ]:
def evaluate_models(models, test_features, test_labels):
    for model in models:
        predictions = model.predict(test_features)
        # Calcular métricas


# 7. Calcular as métricas de Sensibilidade, Especificidade e F1-Score

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

def calculate_metrics(true_labels, predictions):
    cm = confusion_matrix(true_labels, predictions)
    report = classification_report(true_labels, predictions)
    print(report)

# 8.Treinamento com VGG16 e ResNet50 com Data Augmentation
Utilize as redes pré-treinadas com keras e ajuste as camadas fully connected para o problema de 4 classes.

In [ ]:
from tensorflow.keras.applications import ResNet50

# Modelos VGG16 e ResNet50 com Transfer Learning
def build_model(base_model):
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(4, activation='softmax')(x)  # 4 classes
    return Model(inputs=base_model.input, outputs=x)

# Data Augmentation
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
